In [7]:
from PIL import Image
import rasterio
import math
import numpy as np
import rasterio.transform
from bokeh import palettes
import json

### Load up the data

The elevation.tiff file you should get by running the Create Elevation Tiff notebook.

You can download the population tiff from https://sedac.ciesin.columbia.edu/data/set/gpw-v4-population-count-adjusted-to-2015-unwpp-country-totals-rev11

Center for International Earth Science Information Network - CIESIN - Columbia University. 2018. Gridded Population of the World, Version 4 (GPWv4): Population Count Adjusted to Match 2015 Revision of UN WPP Country Totals, Revision 11. Palisades, NY: NASA Socioeconomic Data and Applications Center (SEDAC). https://doi.org/10.7927/H4PN93PB. Accessed 4 september 2021.

In [8]:
pop_tiff = 'gpw_v4_population_count_adjusted_to_2015_unwpp_country_totals_rev11_2020_2pt5_min.tif'
pop = rasterio.open(pop_tiff)
pop_data = pop.read(1)

elevation = rasterio.open('elevation.tiff')
elevation_data = elevation.read(1)

elevation_data.shape, pop.bounds, pop.crs

((4320, 8640),
 BoundingBox(left=-180.0, bottom=-90.0, right=180.0000000000232, top=90.0000000000116),
 CRS.from_epsg(4326))

### Quick check

Let's do some quick calculations on the data to see how many people live below 200m and what the maximum number of
people on a pixel is.

In [9]:
world_population = pop_data[(pop_data > 0)].sum()

pop_data[(elevation_data < 200) & (pop_data > 0)].sum() / world_population, pop_data.max()

(0.5360479, 1646519.8)

### Color setup

We set up some colors based on the inferno pallette. Also pick to types of blue

In [10]:
min_val = 1
max_val = min(10_000, pop_data.max())
logmin = math.log(min_val)
logmax = math.log(max_val)


def rgb(s):
    return np.array([int(s[1:3], 16), int(s[3:5], 16), int(s[5:7], 16)])

pal = [rgb(s) for s in palettes.inferno(20)]
blue = np.array([40, 40, 200])
light_blue = np.array([70, 70, 240])
ocean = np.full(elevation_data.shape + (3,), blue, dtype=np.uint8)
pal, blue

([array([0, 0, 3]),
  array([ 7,  5, 29]),
  array([23, 11, 59]),
  array([46, 10, 90]),
  array([ 69,  10, 105]),
  array([ 91,  17, 110]),
  array([112,  25, 110]),
  array([133,  32, 106]),
  array([155,  40, 100]),
  array([175,  49,  91]),
  array([196,  60,  78]),
  array([213,  73,  64]),
  array([229,  91,  48]),
  array([240, 111,  31]),
  array([247, 133,  14]),
  array([251, 158,   7]),
  array([251, 183,  28]),
  array([246, 211,  63]),
  array([241, 235, 108]),
  array([252, 254, 164])],
 array([ 40,  40, 200]))

### Draw the maps!

For each pixel we pick a color based on whether it is below the cut-off of the sea level, the population living there. Then do this for 9 different levels of sealevel and 🚀

In [11]:
def color_for_value(val, el, cut_off):
    if val < 0:
        return blue
    elif (cut_off is not None and el < cut_off):
        return light_blue
    elif val < min_val:
        return pal[0]
    elif val >= max_val:
        return pal[-1]
    else:
        return pal[int(len(pal) * (math.log(val) - logmin) / logmax)]
    
color_for_value(400, 10, 1)

array([240, 111,  31])

In [12]:
pop_under = {}

for cut_off in -10, 2, 5, 10, 25, 50, 100, 200, 500:
    pop_under[cut_off] = pop_data[(elevation_data < cut_off) & (pop_data > 0)].sum() / 1_000_000
    print(pop_under[cut_off], 'million people live below', cut_off)
    h, w = pop_data.shape
    pop_map = np.zeros((h, w, 3), np.uint8)
    for x in range(w):
        for y in range(h):
            pop_map[y][x] = color_for_value(pop_data[y][x], elevation_data[y][x], cut_off=cut_off)

    img = Image.fromarray(pop_map)
    if cut_off < 0:
        ext = ''
    else:
        ext = '_' + str(cut_off)
    img.save(f'elevation{ext}.png')
    img.resize((w // 20, h // 20), Image.LANCZOS).save(f'elevation_thumb{ext}.jpg')
    
with open('people_under.json', 'w') as fout:
    json.dump(pop_under, fout, indent=2)

7.6407145 million people live below -10
177.264352 million people live below 2
374.025568 million people live below 5
785.290944 million people live below 10
1481.814272 million people live below 25
2200.117504 million people live below 50
3093.371136 million people live below 100
4158.65088 million people live below 200
5768.622592 million people live below 500
